In [4]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# EPICS Proxy example

The epics proxy should provide methods that look the same as those in pyepics
https://cars9.uchicago.edu/software/python/pyepics3/


In [5]:
from lcls_live.epics import epics_proxy
import os

In [7]:
epics = epics_proxy('data/epics_snapshot_2018-03-06T15:21:15.000000-08:00.json', verbose=True)

Loaded data/epics_snapshot_2018-03-06T15:21:15.000000-08:00.json with 2068 PVs


In [8]:
# Data is kept internally as a dict in .pvdata:
list(epics.pvdata)[0:10]

['ACCL:IN20:350:FUDGE',
 'ACCL:LI21:180:L1X_S_AV',
 'ACCL:LI21:180:L1X_S_PV',
 'ACCL:LI21:1:FUDGE',
 'ACCL:LI21:1:L1S_S_PV',
 'ACCL:LI22:1:FUDGE',
 'ACCL:LI24:100:KLY_PDES',
 'ACCL:LI24:200:KLY_PDES',
 'ACCL:LI24:300:KLY_PDES',
 'ACCL:LI25:1:FUDGE']

In [9]:
# caget fetches from .pvdata
epics.caget('KLYS:LI21:11:ENLD')

115.836

In [10]:
# caput writes to .pvdata
epics.caput('KLYS:LI21:11:ENLD', 3)
epics.caget('KLYS:LI21:11:ENLD')

3

In [11]:
# PV
pv = epics.PV('KLYS:LI21:11:ENLD')
pv.get()

PV for KLYS:LI21:11:ENLD


3

In [12]:
# caget_many 
epics.caget_many(['KLYS:LI21:11:ENLD',
 'ACCL:LI21:1:L1S_S_PV',
 'KLYS:LI21:11:PDES',
 'KLYS:LI21:11:SWRD',
 'KLYS:LI21:11:STAT',
 'KLYS:LI21:11:HDSC',
 'KLYS:LI21:11:DSTA',
 'KLYS:LI21:11:BEAMCODE1_STAT',
 'ACCL:LI21:180:L1X_S_AV',
 'ACCL:LI21:180:L1X_S_PV'])

[3,
 -26.12494033014282,
 -24.5,
 0,
 1,
 1048577.0,
 [1610612738.0, 528640.0],
 1.0,
 20.01551950520014,
 -159.9562692633193]

In [13]:
# Non-epics routine to save .pvdata to a JSON file
epics.save(filename='test.json')

Saved test.json


In [14]:
# Corresponding load routine
epics.load(filename='test.json')

Loaded test.json with 2068 PVs


In [15]:
# Cleanup
os.remove('test.json')

# Nested proxies

In [16]:
# Start with a 'base' source of epics data
epics0 = epics_proxy('data/epics_snapshot_2018-03-06T15:21:15.000000-08:00.json', verbose=True)

# Make another proxy
epics1 = epics_proxy('cache.json', epics=epics0, verbose=True)

Loaded data/epics_snapshot_2018-03-06T15:21:15.000000-08:00.json with 2068 PVs


In [17]:
# Get a value
epics1.caget('KLYS:LI21:11:ENLD')

Error: pv not cached: KLYS:LI21:11:ENLD
Loading from epics


115.836

In [18]:
# This is the internal data
epics1.pvdata

{'KLYS:LI21:11:ENLD': 115.836}

In [19]:
epics1.caget_many(['KLYS:LI21:11:ENLD', 'KLYS:LI21:11:DSTA'])

{'KLYS:LI21:11:ENLD': 115.836, 'KLYS:LI21:11:DSTA': [1610612738.0, 528640.0]}

In [20]:
# This is the internal data
epics1.pvdata

{'KLYS:LI21:11:ENLD': 115.836}

In [21]:
# Save to JSON
epics1.save()

Saved cache.json


In [22]:
# Cleanup
os.remove('cache.json')